In [1]:
# import numpy as np
# import pandas as pd
# import gc
# import os
# import hickle as hkl
# from numpy import random
# from sklearn.preprocessing import MinMaxScaler
# import joblib

# from sklearn.preprocessing import LabelEncoder
# # from sklearn.externals import joblib

# from keras.models import Model
# from keras.optimizers import Adam, SGD
# from keras import initializers, regularizers, constraints
# from keras.layers import Bidirectional, GlobalMaxPooling1D, Dense, Embedding
# from keras.layers import concatenate, Input, LSTM, GRU, merge, Lambda, Dot, Add, Multiply, \
# wrappers, Dropout, GlobalAveragePooling1D, Bidirectional, BatchNormalization, Activation
# import tensorflow as tf
# from keras.models import Sequential
# from keras.layers.core import Reshape
# import keras.backend as K
# from keras.engine.topology import Layer
# from keras.layers import LSTM, Dense
# from tqdm import tqdm
# import glob
# from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback, ReduceLROnPlateau

# import keras.backend as KTF
# import tensorflow as tf
# #os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tf.Session(config=config)
# KTF.set_session(sess)

import numpy as np
import pandas as pd
import gc
import os
import hickle as hkl
from numpy import random
from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import LabelEncoder
#from sklearn.externals import joblib
import joblib

from keras.models import Model
#from keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers import Adam, SGD
from keras import initializers, regularizers, constraints
from keras.layers import Bidirectional, GlobalMaxPooling1D, Dense, Embedding
from keras.layers import concatenate, Input, LSTM, GRU, merge, Lambda, Dot, Add, Multiply, \
wrappers, Dropout, GlobalAveragePooling1D, Bidirectional, BatchNormalization, Activation
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Reshape
import keras.backend as K
#from keras.engine.topology import Layer
from tensorflow.keras.layers import Layer
from keras.layers import LSTM, Dense
from tqdm import tqdm
import glob
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback, ReduceLROnPlateau

import keras.backend as KTF
import tensorflow as tf
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#config = tf.ConfigProto()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
KTF.set_session(sess)

# data_path = 'Skip_Data/'
data_path = 'C:/Users/13528/Task 1/Skip Data/'


In [2]:

batch_size = 1500 # skip_net_glove_max_mtsk_more_layer_best

batch_size = 3000 # skip_net_glove_max_mtsk_more_layer_best_v2

batch_size = 3000 # skip_net_glove_max_mtsk_more_layer_best_bn

batch_size = 2200 # skip_net_glove_max_mtsk_more_layer_best_v3

batch_size = 2200 # skip_net_glove_max_mtsk_more_layer_best_v4

# data_path = 'Skip_Data/'
data_path = 'C:/Users/13528/Task 1/Skip Data/'

le_track_id = joblib.load('le_track_id.pkl')

spotify_song_fea = pd.read_parquet('spotify_song_fea.parquet')

le = LabelEncoder()
##
spotify_song_fea['mode'] = le.fit_transform(spotify_song_fea['mode'])

track_id = np.array(spotify_song_fea['track_id'])

cols_to_be_normalized = ['duration', 'release_year', 'us_popularity_estimate',
       'acousticness', 'beat_strength', 'bounciness', 'danceability',
       'dyn_range_mean', 'energy', 'flatness', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mechanism', 'mode', 'organism', 'speechiness',
       'tempo', 'time_signature', 'valence', 'acoustic_vector_0',
       'acoustic_vector_1', 'acoustic_vector_2', 'acoustic_vector_3',
       'acoustic_vector_4', 'acoustic_vector_5', 'acoustic_vector_6',
       'acoustic_vector_7']

scaler = MinMaxScaler()

spotify_song_array = scaler.fit_transform(spotify_song_fea[cols_to_be_normalized])

order = np.argsort(track_id)
spotify_song_array = spotify_song_array[order,:]

song_zero_embedding = -2*np.ones((1,spotify_song_array.shape[1]))
spotify_song_array = np.concatenate((song_zero_embedding,spotify_song_array),axis=0)

In [16]:
'''


batch_size = 1500 # skip_net_glove_max_mtsk_more_layer_best

batch_size = 3000 # skip_net_glove_max_mtsk_more_layer_best_v2

batch_size = 3000 # skip_net_glove_max_mtsk_more_layer_best_bn

data_path = 'Skip_Data/'

le_track_id = joblib.load('le_track_id.pkl')

spotify_song_fea = pd.read_parquet('spotify_song_fea.parquet')

le = LabelEncoder()
##
spotify_song_fea['mode'] = le.fit_transform(spotify_song_fea['mode'])

track_id = np.array(spotify_song_fea['track_id'])

spotify_song_fea['duration'] = spotify_song_fea['duration']/100
spotify_song_fea['release_year'] = spotify_song_fea['release_year']-1950
spotify_song_fea['us_popularity_estimate'] = spotify_song_fea['us_popularity_estimate'] - 90

spotify_song_fea['duration_05'] = spotify_song_fea['duration']**0.5
spotify_song_fea['duration_15'] = spotify_song_fea['duration']**1.5
spotify_song_fea['duration_2'] = spotify_song_fea['duration']**2
spotify_song_fea['duration_3'] = spotify_song_fea['duration']**3

spotify_song_fea['release_year_05'] = spotify_song_fea['release_year']**0.5
spotify_song_fea['release_year_15'] = spotify_song_fea['release_year']**1.5
spotify_song_fea['release_year_2'] = spotify_song_fea['release_year']**2
spotify_song_fea['release_year_3'] = spotify_song_fea['release_year']**3

spotify_song_fea['us_popularity_estimate_15'] = spotify_song_fea['us_popularity_estimate']**1.5
spotify_song_fea['us_popularity_estimate_2'] = spotify_song_fea['us_popularity_estimate']**2
spotify_song_fea['us_popularity_estimate_3'] = spotify_song_fea['us_popularity_estimate']**3

cols_to_be_normalized = ['us_popularity_estimate_3', 'us_popularity_estimate_2', 'us_popularity_estimate_15', 'release_year_05','release_year_15','release_year_2','duration_05','duration_15','duration_2','duration', 'release_year', 'us_popularity_estimate',
       'acousticness', 'beat_strength', 'bounciness', 'danceability',
       'dyn_range_mean', 'energy', 'flatness', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mechanism', 'mode', 'organism', 'speechiness',
       'tempo', 'time_signature', 'valence', 'acoustic_vector_0',
       'acoustic_vector_1', 'acoustic_vector_2', 'acoustic_vector_3',
       'acoustic_vector_4', 'acoustic_vector_5', 'acoustic_vector_6',
       'acoustic_vector_7']

scaler = MinMaxScaler()

spotify_song_array = scaler.fit_transform(spotify_song_fea[cols_to_be_normalized])

order = np.argsort(track_id)
spotify_song_array = spotify_song_array[order,:]

song_zero_embedding = -2*np.ones((1,spotify_song_array.shape[1]))
spotify_song_array = np.concatenate((song_zero_embedding,spotify_song_array),axis=0)

'''

/usr/local/lib/python3.5/dist-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [3]:
# Load Glove Embedding

# spotify_song_array_glove = hkl.load('song_embedding_matrix_150.hkl')
spotify_song_array_glove = joblib.load('song_embedding_matrix_150.pkl')

gc.collect()

spotify_song_array = np.concatenate((spotify_song_array,spotify_song_array_glove),axis=1)

In [4]:
# If the previous steps are done correctly, the embedding matrix should be of size (3706389, 179)

spotify_song_array.shape

(3706389, 179)

In [5]:
train_0 = pd.read_csv('C:/Users/13528/Task 1/train_fold/log_0_20180715_000000000000.csv')
print(train_0.columns)

Index(['session_id', 'session_position', 'session_length', 'track_id_clean',
       'skip_1', 'skip_2', 'skip_3', 'not_skipped', 'context_switch',
       'no_pause_before_play', 'short_pause_before_play',
       'long_pause_before_play', 'hist_user_behavior_n_seekfwd',
       'hist_user_behavior_n_seekback', 'hist_user_behavior_is_shuffle',
       'hour_of_day', 'date', 'premium', 'context_type',
       'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end'],
      dtype='object')


In [6]:
le = LabelEncoder()
train_0['context_type'] = le.fit_transform(train_0['context_type'])
print(train_0['context_type'])

0          2
1          2
2          2
3          2
4          2
          ..
2990604    5
2990605    5
2990606    5
2990607    5
2990608    5
Name: context_type, Length: 2990609, dtype: int32


In [7]:
len(train_0['context_type'].unique()) # context  = 6

6

In [8]:
train_0['hist_user_behavior_reason_start'] = le.fit_transform(train_0['hist_user_behavior_reason_start'])
print(train_0['hist_user_behavior_reason_start']) 

0          8
1          8
2          8
3          8
4          8
          ..
2990604    4
2990605    1
2990606    4
2990607    1
2990608    4
Name: hist_user_behavior_reason_start, Length: 2990609, dtype: int32


In [9]:
len(train_0['hist_user_behavior_reason_start'].unique()) #reason start = 12/11

11

In [10]:
train_0['hist_user_behavior_reason_end'] = le.fit_transform(train_0['hist_user_behavior_reason_end'])
print(train_0['hist_user_behavior_reason_end']) 

0          8
1          8
2          8
3          8
4          8
          ..
2990604    1
2990605    4
2990606    1
2990607    4
2990608    4
Name: hist_user_behavior_reason_end, Length: 2990609, dtype: int32


In [11]:
len(train_0['hist_user_behavior_reason_start'].unique()) #reason end=11

11

In [12]:
le_context_type = train_0['context_type']
le_reason_start = train_0['hist_user_behavior_reason_start']
le_reason_end = train_0['hist_user_behavior_reason_end']

In [13]:
joblib.dump(le_context_type, data_path+'le_context_type.pkl')
joblib.dump(le_reason_start, data_path+'le_reason_start.pkl')
joblib.dump(le_reason_end, data_path+'le_reason_end.pkl')

['C:/Users/13528/Task 1/Skip Data/le_reason_end.pkl']

In [14]:
le_context_type.unique()

array([2, 5, 4, 3, 0, 1])

In [15]:
#from sklearn.externals import joblib
#joblib.dump(le_context_type, data_path+'le_context_type.pkl')
#joblib.dump(le_reason_start, data_path+'le_reason_start.pkl')
#joblib.dump(le_reason_end, data_path+'le_reason_end.pkl')

['Skip_Data/le_reason_end.pkl']

In [15]:
le_context_type = joblib.load(data_path+'le_context_type.pkl')
le_reason_start = joblib.load(data_path+'le_reason_start.pkl')
le_reason_end = joblib.load(data_path+'le_reason_end.pkl')

# n_context_type = len(le_context_type.classes_) + 1
# n_reason_start = len(le_reason_start.classes_) + 1
# n_reason_end = len(le_reason_end.classes_) + 1
n_context_type = len(le_context_type.unique())
n_reason_start = len(le_reason_start.unique())
n_reason_end = len(le_reason_end.unique())

In [22]:
temp = pd.read_parquet(train_files[0])

In [23]:
temp

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,0,1,17,1310247,False,False,True,False,0,0,...,0,0,0,False,14,2018-07-23,True,editorial_playlist,appload,endplay
1,0,2,17,3649627,True,True,True,False,1,1,...,0,0,0,True,14,2018-07-23,True,editorial_playlist,clickrow,fwdbtn
2,0,3,17,3272426,True,True,True,False,0,1,...,0,0,0,True,14,2018-07-23,True,editorial_playlist,fwdbtn,fwdbtn
3,0,4,17,2511613,True,True,True,False,0,1,...,0,0,0,True,14,2018-07-23,True,editorial_playlist,fwdbtn,fwdbtn
4,0,5,17,3472814,True,True,True,False,0,1,...,0,0,0,True,14,2018-07-23,True,editorial_playlist,fwdbtn,fwdbtn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3276791,196334,10,14,2280589,False,False,False,True,0,1,...,0,0,0,False,18,2018-07-23,True,user_collection,backbtn,trackdone
3276792,196334,11,14,1644516,False,False,False,True,0,1,...,0,0,0,False,18,2018-07-23,True,user_collection,trackdone,trackdone
3276793,196334,12,14,289091,False,False,False,True,0,1,...,0,0,0,False,19,2018-07-23,True,user_collection,trackdone,trackdone
3276794,196334,13,14,772842,False,False,False,True,0,0,...,1,0,0,False,19,2018-07-23,True,user_collection,trackdone,trackdone


In [27]:
# temp['context_type'] = le_context_type.transform(temp['context_type'])
le_context_type

0          2
1          2
2          2
3          2
4          2
          ..
2990604    5
2990605    5
2990606    5
2990607    5
2990608    5
Name: context_type, Length: 2990609, dtype: int32

In [32]:
test1 = le.transform(temp['context_type'])

In [34]:
test1

array([2, 2, 2, ..., 5, 5, 5])

In [43]:
temp['hist_user_behavior_reason_start']

0            appload
1           clickrow
2             fwdbtn
3             fwdbtn
4             fwdbtn
             ...    
3276791      backbtn
3276792    trackdone
3276793    trackdone
3276794    trackdone
3276795    trackdone
Name: hist_user_behavior_reason_start, Length: 3276796, dtype: object

In [45]:
test2 = le.fit_transform(temp['hist_user_behavior_reason_start'])

In [53]:
df = temp
if_gen = 0
batch_size = 1500
sample = 0

In [55]:
df['date'] = pd.to_datetime(df['date'] )
print(df['date'])

0         2018-07-23
1         2018-07-23
2         2018-07-23
3         2018-07-23
4         2018-07-23
             ...    
3276791   2018-07-23
3276792   2018-07-23
3276793   2018-07-23
3276794   2018-07-23
3276795   2018-07-23
Name: date, Length: 3276796, dtype: datetime64[ns]


In [57]:
df['date_gap'] = df['date'] - pd.Timestamp(2018, 7, 13)
df['date_gap'] = df['date_gap'].dt.days
print(df['date_gap'])

0          10
1          10
2          10
3          10
4          10
           ..
3276791    10
3276792    10
3276793    10
3276794    10
3276795    10
Name: date_gap, Length: 3276796, dtype: int64


In [58]:
df['context_type'] = le.fit_transform(df['context_type'])
print(df['context_type'])

0          2
1          2
2          2
3          2
4          2
          ..
3276791    5
3276792    5
3276793    5
3276794    5
3276795    5
Name: context_type, Length: 3276796, dtype: int32


In [59]:
df['hist_user_behavior_reason_start'] = le.fit_transform(df['hist_user_behavior_reason_start'])
print(df['hist_user_behavior_reason_start'])

0          0
1          2
2          4
3          4
4          4
          ..
3276791    1
3276792    8
3276793    8
3276794    8
3276795    8
Name: hist_user_behavior_reason_start, Length: 3276796, dtype: int32


In [60]:
df['hist_user_behavior_reason_end'] = le.fit_transform(df['hist_user_behavior_reason_end'])
print(df['hist_user_behavior_reason_end'])

0          3
1          4
2          4
3          4
4          4
          ..
3276791    8
3276792    8
3276793    8
3276794    8
3276795    8
Name: hist_user_behavior_reason_end, Length: 3276796, dtype: int32


In [61]:
session_cols = ['session_id', 'session_position', 'session_length', 'track_id_clean',
                    'skip_1', 'skip_2', 'skip_3', 'not_skipped', 'context_switch',
                    'no_pause_before_play', 'short_pause_before_play',
                    'long_pause_before_play', 'hist_user_behavior_n_seekfwd',
                    'hist_user_behavior_n_seekback', 'hist_user_behavior_is_shuffle',
                    'hour_of_day', 'premium', 'context_type',
                    'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end','date_gap']

In [62]:
session_id = np.unique(df['session_id'])
print(session_id)

[     0      1      2 ... 196332 196333 196334]


In [64]:
session_position_offset = np.random.choice(elements, len(session_id), p=probabilities)
print(session_position_offset)

[-1  2  1 ... -1 -1 -1]


In [65]:
df_offset = pd.DataFrame()

In [67]:
df_offset['session_id'] = session_id

In [69]:
if sample == 1:
    df_offset['session_position_offset'] = session_position_offset
else:
    df_offset['session_position_offset'] = 0

In [71]:
n0 = df.shape[0]
print(n0)

3276796


In [73]:
df = pd.merge(df, df_offset, on = 'session_id', how = 'left')

In [75]:
df['session_position'] = df['session_position'] + df['session_position_offset']
df = df[df['session_position']<=df['session_length']].reset_index(drop=True)
df = df[df['session_position']>=1].reset_index(drop=True)

In [77]:
n1 = df.shape[0]
print('shrinkage:',n0, n1)

shrinkage: 3276796 3276796


In [78]:
df['track_id_clean'] = df['track_id_clean'] + 1
df['hour_of_day'] = df['hour_of_day']/24
print(df['track_id_clean'])
print(df['hour_of_day'])

0          1310248
1          3649628
2          3272427
3          2511614
4          3472815
            ...   
3276791    2280590
3276792    1644517
3276793     289092
3276794     772843
3276795    1017203
Name: track_id_clean, Length: 3276796, dtype: int32
0          0.583333
1          0.583333
2          0.583333
3          0.583333
4          0.583333
             ...   
3276791    0.750000
3276792    0.750000
3276793    0.791667
3276794    0.791667
3276795    0.791667
Name: hour_of_day, Length: 3276796, dtype: float64


In [79]:
raw_data = np.array(df[session_cols].values) * 1
raw_data = raw_data.astype(np.float)

C:\Users\13528\AppData\Local\Temp/ipykernel_25132/3009888740.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_data = raw_data.astype(np.float)


In [81]:
len(raw_data)

3276796

In [83]:
input_raw_data = raw_data[raw_data[:,1] * 2 <= raw_data[:, 2], :]
output_raw_data = raw_data[raw_data[:,1] * 2 > raw_data[:, 2], :]

input_raw_data_final_p = np.floor(input_raw_data[:, 2] / 2).astype(np.int)
input_raw_data_final_p = 10 - input_raw_data_final_p + input_raw_data[:, 1]

output_raw_data_first_p = np.floor(output_raw_data[:, 2] / 2).astype(np.int)
output_raw_data_first_p = -output_raw_data_first_p + output_raw_data[:, 1]

n_session = int(np.max(raw_data[:,0])) + 1
gc.collect()
input_data = -2 * np.ones((n_session * 10, 18))
output_data = -2 * np.ones((n_session * 10, 19))

input_data[:, 17] = n_reason_end - 1
output_data[:, 17] = n_reason_end - 1
    
input_data[:, 16] = n_reason_start - 1
output_data[:, 16] = n_reason_start - 1
    
input_data[:, 15] = n_context_type - 1
output_data[:, 15] = n_context_type - 1
    
'''    
output_premium = input_data[:,9,14]    
output_premium = np.reshape(output_premium,(len(output_premium),1,1))
output_premium = np.tile(output_premium,(1,10,1))
'''
    
input_raw_data[:,2] = input_raw_data[:,1]/input_raw_data[:,2]
output_raw_data[:,2] = output_raw_data[:,1]/output_raw_data[:,2]


input_data[input_raw_data[:, 0].astype(np.int) * 10 + input_raw_data_final_p.astype(np.int) - 1,
:] = input_raw_data[:, 2:20]
output_data[output_raw_data[:, 0].astype(np.int) * 10 + output_raw_data_first_p.astype(np.int) - 1,
:] = output_raw_data[:, 2:21]
input_data = np.reshape(input_data, (n_session, 10, input_data.shape[1]))
output_data = np.reshape(output_data, (n_session, 10, output_data.shape[1]))
output_data_target = output_data[:, :, 2:13]
output_data_target[output_data_target>1] = 1
output_data_target[output_data_target < 0] = 0
    
output_date_id = output_data[:, :, 18]
    
#output_data_target[:,:,3] = 1 - output_data_target[:,:,1]   
    
output_data_id = output_data[:,:,1]

order = np.arange(n_session)
random.shuffle(order)
    
if if_gen == 1:
    order = order[0:int(batch_size*np.floor(n_session/batch_size))]        

input_data = input_data[order,:,:]  
 
input_data_context_id = input_data[:,:,15]
input_data_start_id = input_data[:,:,16]
input_data_end_id = input_data[:,:,17]

input_data = input_data[:,:,range(15)]

input_data_id = input_data[:,:,1]
input_data = np.delete(input_data, 1, 2)
output_data_id = output_data_id[order,:]
output_data_target = output_data_target[order,:]
    
output_data = output_data[:,:,0]
    
input_data_context_id[input_data_context_id<0] = 0
input_data_start_id[input_data_start_id<0] = 0
input_data_end_id[input_data_end_id<0] = 0
input_data_id[input_data_id<0] = 0
output_data_id[output_data_id<0] = 0
    
input_data_context_id = input_data_context_id.astype(int)
input_data_start_id = input_data_start_id.astype(int)
input_data_end_id = input_data_end_id.astype(int)
input_data_id = input_data_id.astype(int)
output_data_id = output_data_id.astype(int)

C:\Users\13528\AppData\Local\Temp/ipykernel_25132/1748667486.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  input_raw_data_final_p = np.floor(input_raw_data[:, 2] / 2).astype(np.int)
C:\Users\13528\AppData\Local\Temp/ipykernel_25132/1748667486.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review y

In [86]:
input_data

array([[[-2.        , -2.        , -2.        , ..., -2.        ,
         -2.        , -2.        ],
        [-2.        , -2.        , -2.        , ..., -2.        ,
         -2.        , -2.        ],
        [ 0.0625    ,  0.        ,  0.        , ...,  0.        ,
          0.45833333,  1.        ],
        ...,
        [ 0.375     ,  0.        ,  0.        , ...,  0.        ,
          0.45833333,  1.        ],
        [ 0.4375    ,  0.        ,  0.        , ...,  0.        ,
          0.45833333,  1.        ],
        [ 0.5       ,  0.        ,  0.        , ...,  0.        ,
          0.5       ,  1.        ]],

       [[-2.        , -2.        , -2.        , ..., -2.        ,
         -2.        , -2.        ],
        [-2.        , -2.        , -2.        , ..., -2.        ,
         -2.        , -2.        ],
        [ 0.05882353,  0.        ,  0.        , ...,  1.        ,
          0.29166667,  1.        ],
        ...,
        [ 0.35294118,  0.        ,  0.        , ...,  

In [87]:
output_data

array([[ 0.52941176,  0.58823529,  0.64705882, ...,  0.94117647,
         1.        , -2.        ],
       [ 0.55      ,  0.6       ,  0.65      , ...,  0.9       ,
         0.95      ,  1.        ],
       [ 0.53846154,  0.61538462,  0.69230769, ..., -2.        ,
        -2.        , -2.        ],
       ...,
       [ 0.55      ,  0.6       ,  0.65      , ...,  0.9       ,
         0.95      ,  1.        ],
       [ 0.55      ,  0.6       ,  0.65      , ...,  0.9       ,
         0.95      ,  1.        ],
       [ 0.57142857,  0.64285714,  0.71428571, ..., -2.        ,
        -2.        , -2.        ]])

In [88]:
input_data_context_id

array([[5, 5, 5, ..., 5, 5, 5],
       [5, 5, 0, ..., 0, 0, 0],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [5, 5, 5, ..., 0, 0, 0],
       [5, 5, 5, ..., 2, 2, 2],
       [5, 5, 5, ..., 1, 1, 1]])

In [89]:
 input_data_start_id

array([[10, 10,  0, ...,  8,  8,  8],
       [10, 10,  2, ...,  8,  4,  4],
       [ 8,  4,  2, ...,  4,  8,  8],
       ...,
       [10, 10, 10, ...,  1,  4,  8],
       [10, 10, 10, ...,  2,  4,  4],
       [10, 10, 10, ...,  4,  4,  2]])

In [90]:
 input_data_end_id

array([[9, 9, 1, ..., 8, 8, 8],
       [9, 9, 8, ..., 4, 4, 4],
       [4, 3, 4, ..., 8, 8, 4],
       ...,
       [9, 9, 9, ..., 4, 8, 1],
       [9, 9, 9, ..., 4, 4, 4],
       [9, 9, 9, ..., 4, 3, 3]])

In [91]:
input_data_id

array([[      0,       0, 2187372, ..., 1296440, 3548333, 1812033],
       [      0,       0,  154893, ..., 2971950, 1868581, 2980961],
       [1859551, 3322900, 2020951, ..., 1727374, 1210450, 1088891],
       ...,
       [      0,       0,       0, ..., 1871425,  385081, 3198255],
       [      0,       0,       0, ...,   40311, 2367471, 1641216],
       [      0,       0,       0, ..., 2489144, 1731912, 3653680]])

In [92]:
 output_data_id

array([[2187372,  199246, 1005958, ..., 1485702,       0,       0],
       [1731214, 2577657, 3272200, ..., 2980961, 1868581,       0],
       [1661821,  216627,  271471, ..., 1579802,  445820, 3048804],
       ...,
       [ 385081, 3198255,  385081, ...,       0,       0,       0],
       [3189680, 2481919,    7330, ...,  169104,       0,       0],
       [1259627,  216627,  430098, ...,       0,       0,       0]])

In [93]:
output_data_target

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 0., 0., 1.],
        [1., 1., 1., ..., 0., 0., 1.],
        [1., 1., 1., ..., 0., 0., 1.],
        ...,
        [0., 1., 1., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 0., 0., 1.],
        [1., 1., 1., ..., 0., 0., 1.],
        [0., 1., 1., ..., 0., 0., 1.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 1., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 1., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [94]:
output_date_id

array([[10., 10., 10., ..., 10., 10., -2.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., -2., -2., -2.],
       ...,
       [ 9.,  9.,  9., ...,  9.,  9.,  9.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., -2., -2., -2.]])

In [70]:
df_offset

,session_id,session_position_offset
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
196330,196330,0
196331,196331,0
196332,196332,0
196333,196333,0


In [76]:
df

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end,date_gap,session_position_offset
0,0,1,17,1310247,False,False,True,False,0,0,...,0,False,14,2018-07-23,True,2,0,3,10,0
1,0,2,17,3649627,True,True,True,False,1,1,...,0,True,14,2018-07-23,True,2,2,4,10,0
2,0,3,17,3272426,True,True,True,False,0,1,...,0,True,14,2018-07-23,True,2,4,4,10,0
3,0,4,17,2511613,True,True,True,False,0,1,...,0,True,14,2018-07-23,True,2,4,4,10,0
4,0,5,17,3472814,True,True,True,False,0,1,...,0,True,14,2018-07-23,True,2,4,4,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3276791,196334,10,14,2280589,False,False,False,True,0,1,...,0,False,18,2018-07-23,True,5,1,8,10,0
3276792,196334,11,14,1644516,False,False,False,True,0,1,...,0,False,18,2018-07-23,True,5,8,8,10,0
3276793,196334,12,14,289091,False,False,False,True,0,1,...,0,False,19,2018-07-23,True,5,8,8,10,0
3276794,196334,13,14,772842,False,False,False,True,0,0,...,0,False,19,2018-07-23,True,5,8,8,10,0


In [16]:
#For v5

elements = [-2,-1,0,1,2]
probabilities = [0.1,0.25,0.3,0.25,0.1]
#np.random.choice(elements, 10, p=probabilities)

def ProcessSessionCate(df, if_gen = 0, batch_size = batch_size, sample = 0):
    df['date'] = pd.to_datetime(df['date'] )
        
    df['date_gap'] = df['date'] - pd.Timestamp(2018, 7, 13)
    df['date_gap'] = df['date_gap'].dt.days
    
    
#     df['context_type'] = le_context_type.transform(df['context_type'])
#     df['hist_user_behavior_reason_start'] = le_reason_start.transform(df['hist_user_behavior_reason_start'])
#     df['hist_user_behavior_reason_end'] = le_reason_end.transform(df['hist_user_behavior_reason_end'])
    df['context_type'] = le.fit_transform(df['context_type'])
    df['hist_user_behavior_reason_start'] = le.fit_transform(df['hist_user_behavior_reason_start'])
    df['hist_user_behavior_reason_end'] = le.fit_transform(df['hist_user_behavior_reason_end'])
    session_cols = ['session_id', 'session_position', 'session_length', 'track_id_clean',
                    'skip_1', 'skip_2', 'skip_3', 'not_skipped', 'context_switch',
                    'no_pause_before_play', 'short_pause_before_play',
                    'long_pause_before_play', 'hist_user_behavior_n_seekfwd',
                    'hist_user_behavior_n_seekback', 'hist_user_behavior_is_shuffle',
                    'hour_of_day', 'premium', 'context_type',
                    'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end','date_gap']
    
    session_id = np.unique(df['session_id'])    
    session_position_offset = np.random.choice(elements, len(session_id), p=probabilities)
    df_offset = pd.DataFrame()
    df_offset['session_id'] = session_id
    if sample == 1:
        df_offset['session_position_offset'] = session_position_offset
    else:
        df_offset['session_position_offset'] = 0
        
    n0 = df.shape[0]
        
    df = pd.merge(df, df_offset, on = 'session_id', how = 'left')
    df['session_position'] = df['session_position'] + df['session_position_offset']
    df = df[df['session_position']<=df['session_length']].reset_index(drop=True)
    df = df[df['session_position']>=1].reset_index(drop=True)
    
    n1 = df.shape[0]
    
    #print('shrinkage:',n0, n1)

    df['track_id_clean'] = df['track_id_clean'] + 1
    df['hour_of_day'] = df['hour_of_day']/24

    raw_data = np.array(df[session_cols].values) * 1
    raw_data = raw_data.astype(np.float)

    input_raw_data = raw_data[raw_data[:,1] * 2 <= raw_data[:, 2], :]
    output_raw_data = raw_data[raw_data[:,1] * 2 > raw_data[:, 2], :]

    input_raw_data_final_p = np.floor(input_raw_data[:, 2] / 2).astype(np.int)
    input_raw_data_final_p = 10 - input_raw_data_final_p + input_raw_data[:, 1]

    output_raw_data_first_p = np.floor(output_raw_data[:, 2] / 2).astype(np.int)
    output_raw_data_first_p = -output_raw_data_first_p + output_raw_data[:, 1]

    n_session = int(np.max(raw_data[:,0])) + 1
    gc.collect()
    input_data = -2 * np.ones((n_session * 10, 18))
    output_data = -2 * np.ones((n_session * 10, 19))

    input_data[:, 17] = n_reason_end - 1
    output_data[:, 17] = n_reason_end - 1
    
    input_data[:, 16] = n_reason_start - 1
    output_data[:, 16] = n_reason_start - 1
    
    input_data[:, 15] = n_context_type - 1
    output_data[:, 15] = n_context_type - 1
    
    '''    
    output_premium = input_data[:,9,14]    
    output_premium = np.reshape(output_premium,(len(output_premium),1,1))
    output_premium = np.tile(output_premium,(1,10,1))
    '''
    
    input_raw_data[:,2] = input_raw_data[:,1]/input_raw_data[:,2]
    output_raw_data[:,2] = output_raw_data[:,1]/output_raw_data[:,2]


    input_data[input_raw_data[:, 0].astype(np.int) * 10 + input_raw_data_final_p.astype(np.int) - 1,
    :] = input_raw_data[:, 2:20]
    output_data[output_raw_data[:, 0].astype(np.int) * 10 + output_raw_data_first_p.astype(np.int) - 1,
    :] = output_raw_data[:, 2:21]
    input_data = np.reshape(input_data, (n_session, 10, input_data.shape[1]))
    output_data = np.reshape(output_data, (n_session, 10, output_data.shape[1]))
    output_data_target = output_data[:, :, 2:13]
    output_data_target[output_data_target>1] = 1
    output_data_target[output_data_target < 0] = 0
    
    output_date_id = output_data[:, :, 18]
    
    #output_data_target[:,:,3] = 1 - output_data_target[:,:,1]   
    
    output_data_id = output_data[:,:,1]

    order = np.arange(n_session)
    random.shuffle(order)
    
    if if_gen == 1:
        order = order[0:int(batch_size*np.floor(n_session/batch_size))]        

    input_data = input_data[order,:,:]  
 
    input_data_context_id = input_data[:,:,15]
    input_data_start_id = input_data[:,:,16]
    input_data_end_id = input_data[:,:,17]

    input_data = input_data[:,:,range(15)]

    input_data_id = input_data[:,:,1]
    input_data = np.delete(input_data, 1, 2)
    output_data_id = output_data_id[order,:]
    output_data_target = output_data_target[order,:]
    
    output_data = output_data[:,:,0]
    
    input_data_context_id[input_data_context_id<0] = 0
    input_data_start_id[input_data_start_id<0] = 0
    input_data_end_id[input_data_end_id<0] = 0
    input_data_id[input_data_id<0] = 0
    output_data_id[output_data_id<0] = 0
    
    input_data_context_id = input_data_context_id.astype(int)
    input_data_start_id = input_data_start_id.astype(int)
    input_data_end_id = input_data_end_id.astype(int)
    input_data_id = input_data_id.astype(int)
    output_data_id = output_data_id.astype(int)

    return input_data, output_data, input_data_context_id, input_data_start_id, input_data_end_id,\
           input_data_id, output_data_id, output_data_target, output_date_id

In [17]:
# Specify the train and validation files.
train_files = glob.glob(data_path + '*.csv.parquet')
len(train_files)
valid_files = [data_path +'log_0_20180715_000000000000.csv.parquet']
train_files = list(set(train_files)-set(valid_files))
len(train_files)

10

In [18]:
train_files

['C:/Users/13528/Task 1/Skip Data\\log_0_20180716_000000000000.csv.parquet',
 'C:/Users/13528/Task 1/Skip Data\\log_0_20180722_000000000000.csv.parquet',
 'C:/Users/13528/Task 1/Skip Data\\log_0_20180720_000000000000.csv.parquet',
 'C:/Users/13528/Task 1/Skip Data\\log_0_20180719_000000000000.csv.parquet',
 'C:/Users/13528/Task 1/Skip Data\\log_0_20180721_000000000000.csv.parquet',
 'C:/Users/13528/Task 1/Skip Data\\log_0_20180715_000000000000.csv.parquet',
 'C:/Users/13528/Task 1/Skip Data\\log_0_20180724_000000000000.csv.parquet',
 'C:/Users/13528/Task 1/Skip Data\\log_0_20180717_000000000000.csv.parquet',
 'C:/Users/13528/Task 1/Skip Data\\log_0_20180723_000000000000.csv.parquet',
 'C:/Users/13528/Task 1/Skip Data\\log_0_20180718_000000000000.csv.parquet']

In [19]:
valid_files

['C:/Users/13528/Task 1/Skip Data/log_0_20180715_000000000000.csv.parquet']

In [20]:
def train_generate(batch_size=batch_size, shuffle=True):
    while 1:
        c = [ i for i in range(len(train_files))]
        if shuffle:
            random.shuffle(c)
        j = 0
        X_train_i = []
        Y_train_i = []
        
        for i in c:
            #print(train_files[i])
            
            tmp_data = pd.read_parquet(train_files[i])
            input_data_i, output_data_i, input_data_context_id_i, input_data_start_id_i, input_data_end_id_i, \
            input_data_id_i, output_data_id_i, output_data_target_i, output_date_id_i = ProcessSessionCate(tmp_data,if_gen = 1, batch_size = batch_size, sample = 1)      
            
            n_round = int(input_data_i.shape[0]/batch_size)            
            
            for j in range(n_round):           
                X_train_i = {
                           'context': input_data_context_id_i[j * batch_size:(j + 1) * batch_size],
                            'start': input_data_start_id_i[j * batch_size:(j + 1) * batch_size],
                            'end': input_data_end_id_i[j * batch_size:(j + 1) * batch_size],
                            'input_fea': input_data_i[j * batch_size:(j + 1) * batch_size],
                             'output_fea': output_data_i[j * batch_size:(j + 1) * batch_size],
                            'input_id': input_data_id_i[j * batch_size:(j + 1) * batch_size],
                            'output_id': output_data_id_i[j * batch_size:(j + 1) * batch_size],
                        }                
                
                Y_train_i = output_data_target_i[j * batch_size:(j + 1) * batch_size]
                
                yield (X_train_i,Y_train_i)

In [21]:
# Constructing the validation set.

count = 0
for file in valid_files:
    tmp_data = pd.read_parquet(file)
    input_data_i, output_data_i, input_data_context_id_i, input_data_start_id_i, input_data_end_id_i, \
    input_data_id_i, output_data_id_i, output_data_target_i, output_date_id_i = ProcessSessionCate(tmp_data)
    
    if count == 0:
        valid_input = input_data_i
        valid_output = output_data_i
        valid_input_id = input_data_id_i
        valid_input_data_context_id = input_data_context_id_i
        valid_input_data_start_id = input_data_start_id_i
        valid_input_data_end_id = input_data_end_id_i
        valid_output_id = output_data_id_i
        valid_target = output_data_target_i
    else:
        valid_input = np.concatenate((valid_input,input_data_i),axis=0)
        valid_output = np.concatenate((valid_output,output_data_i),axis=0)
        valid_input_data_context_id = np.concatenate((valid_input_data_context_id,input_data_context_id_i),axis=0)
        valid_input_data_start_id = np.concatenate((valid_input_data_start_id,input_data_start_id_i),axis=0)
        valid_input_data_end_id = np.concatenate((valid_input_data_end_id,input_data_end_id_i),axis=0)
        valid_input_id = np.concatenate((valid_input_id,input_data_id_i),axis=0)
        valid_output_id = np.concatenate((valid_output_id,output_data_id_i),axis=0)
        valid_target = np.concatenate((valid_target,output_data_target_i),axis=0)

    count  = count + 1
    
input_fea_dim = valid_input.shape[2]

C:\Users\13528\AppData\Local\Temp/ipykernel_11360/2914680234.py:52: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_data = raw_data.astype(np.float)
C:\Users\13528\AppData\Local\Temp/ipykernel_11360/2914680234.py:57: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdoc

In [22]:
# Constructing the validation set (cont.)

X_valid = {
   'context': valid_input_data_context_id,
    'start': valid_input_data_start_id,
    'end': valid_input_data_end_id,
    'input_fea': valid_input,
     'output_fea': valid_output,
    'input_id': valid_input_id,
    'output_id': valid_output_id,
}

Y_valid = valid_target

#hkl.dump(Y_valid, data_path+'Y_valid.hkl', mode='w', compression='gzip')

#hkl.dump(valid_output, data_path+'valid_output.hkl', mode='w', compression='gzip')

In [ ]:
'''
def skip_model_5_mtsk_att(cell_size = 20):

    input_data_context_id = Input(shape=[10], name="context")
    input_data_start_id = Input(shape=[10], name="start")
    input_data_end_id = Input(shape=[10], name="end")
    input_data = Input(shape=[10,input_fea_dim], name="input_fea")
    output_data_i = Input(shape=[10], name="output_fea")
    input_id = Input(shape=[10], name="input_id")
    output_id = Input(shape=[10], name="output_id")

    song_emb_layer = Embedding(
        input_dim=spotify_song_array.shape[0],
        output_dim=spotify_song_array.shape[1],
        weights=[spotify_song_array],
        trainable=False
    )

    context_emb_layer = Embedding(n_context_type,20)
    reason_start_emb_layer = Embedding(n_reason_start,20)
    reason_end_emb_layer = Embedding(n_reason_end,20)

    emb_input_id = song_emb_layer(input_id)
    emb_output_id = song_emb_layer(output_id)
    emb_input_data_context_id = context_emb_layer(input_data_context_id)
    emb_input_data_start_id = reason_start_emb_layer(input_data_start_id)
    emb_input_data_end_id = reason_end_emb_layer(input_data_end_id)

    input_data_a = concatenate([input_data,emb_input_id,emb_input_data_context_id,
                              emb_input_data_start_id,emb_input_data_end_id])

    # rnn layers
#    encoder_outputs, state_h
    
    encoder_outputs, rnn_layer = GRU(cell_size, return_sequences=True, return_state=True) (input_data_a)
    
    rnn_layer_last = Lambda(slice_last)(input_data_a)      
#    print(encoder_outputs.shape)
    print(rnn_layer.shape)
#    print(rnn_layer.shape)

    
    rnn_layer = concatenate([rnn_layer, rnn_layer_last])
    
    print(rnn_layer.shape)
    
    # output_data = Reshape([10,1])(output_data_i)
    
    output_data = concatenate([output_data_i, emb_output_id])

    output_data = Dense(spotify_song_array.shape[1]+input_fea_dim+cell_size+60, activation='relu')(output_data)

    rnn_layer_multi = Multiply()([output_data, rnn_layer])
    
    rnn_layer_reshape = Reshape([1,spotify_song_array.shape[1]+input_fea_dim+cell_size+60])(rnn_layer)
    
    rnn_layer_reshape = Lambda(tile_tile)(rnn_layer_reshape)

    output_result = concatenate([rnn_layer_reshape, rnn_layer_multi, output_data])
    
    output_rnn_layer = Bidirectional(GRU(cell_size, return_sequences=True))(output_result)
    
    output_result = concatenate([output_result, output_rnn_layer])

    output_result = Dense(1024, activation='elu')(output_result)
    output_result_1 = Dense(784, activation='elu')(output_result)
    #output_result_2 = Dense(784, activation='elu')(output_result_1)
    
    #output_result = concatenate([output_result, output_result_1, output_result_2])
    
    output_result = Dropout(0.2)(output_result_1)

    output_result = Dense(8, activation='sigmoid')(output_result)
    
    #output_result = Reshape([10])(output_result)


    model = Model(inputs=[input_data_context_id, input_data_start_id,
                          input_data_end_id, input_data, output_data_i, input_id, output_id], outputs=output_result)

    sgd = Adam(lr=0.0005)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['binary_crossentropy', 'binary_accuracy'])
    model.summary()
    return model

checkpoint = ModelCheckpoint('Data/skip_net_glove_max_mtsk_lgb_best.hdf5', monitor='val_binary_accuracy', verbose=1, save_best_only=True, mode='auto')

K.clear_session()

model = skip_model_5_mtsk_att()


In [25]:
# Model Fitting

def tile_tile(X):
    X = K.tile(X, [1,10,1]) 
    return X


def exp_smooth5(X):   
    weight = np.power(0.5,9 - np.arange(10))
    weight = weight/np.sum(weight)
    weight = np.reshape(weight, (1,10,1))
    att_output_weight = K.variable(weight)
    print(att_output_weight.shape)
    att_output_result = X*att_output_weight    
    print(X.shape)
    att_output_result = K.mean(att_output_result, axis=1)
    
    return att_output_result

def exp_smooth8(X):   
    weight = np.power(0.8,9 - np.arange(10))
    weight = weight/np.sum(weight)
    weight = np.reshape(weight, (1,10,1))
    att_output_weight = K.variable(weight)
    print(att_output_weight.shape)
    att_output_result = X*att_output_weight    
    print(X.shape)
    att_output_result = K.mean(att_output_result, axis=1)
    
    return att_output_result


class mstk_ndcg_callback(Callback):
    def __init__(self):
        return

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):      

        y_pred = self.model.predict(X_valid,batch_size=800)
        
        hkl.dump(y_pred, data_path+'y_pred.hkl', mode='w', compression='gzip')
        
        y_pred = y_pred[:,:,1]
        
        Y_valid_i = Y_valid[:,:,1]
        
        Y_pred = 0*y_pred
        Y_pred[y_pred>=0.5] = 1
        
        Y_pred = (Y_pred==Y_valid_i)
        
        Y_pred = Y_pred[valid_output>=0]
        
        #print(Y_pred.shape)

        print(np.mean(Y_pred))
        
        #hkl.dump(song_embedding_matrix, data_path+'song_embedding_matrix_80.hkl', mode='w', compression='gzip')

        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return


def skip_model_5_mtsk_att(cell_size = 350):

    input_data_context_id = Input(shape=[10], name="context")
    input_data_start_id = Input(shape=[10], name="start")
    input_data_end_id = Input(shape=[10], name="end")
    input_data = Input(shape=[10,input_fea_dim], name="input_fea")
    output_data_i = Input(shape=[10], name="output_fea")
    input_id = Input(shape=[10], name="input_id")
    output_id = Input(shape=[10], name="output_id")

    song_emb_layer = Embedding(
        input_dim=spotify_song_array.shape[0],
        output_dim=spotify_song_array.shape[1],
        weights=[spotify_song_array],
        trainable=False
    )

    context_emb_layer = Embedding(n_context_type,25)
    reason_start_emb_layer = Embedding(n_reason_start,25)
    reason_end_emb_layer = Embedding(n_reason_end,25)

    emb_input_id = song_emb_layer(input_id)
    emb_output_id = song_emb_layer(output_id)
    emb_input_data_context_id = context_emb_layer(input_data_context_id)
    emb_input_data_start_id = reason_start_emb_layer(input_data_start_id)
    emb_input_data_end_id = reason_end_emb_layer(input_data_end_id)

    input_data_a = concatenate([input_data,emb_input_id,emb_input_data_context_id,
                              emb_input_data_start_id,emb_input_data_end_id])

    # rnn layers
#    encoder_outputs, state_h
    
    encoder_outputs, rnn_layer = GRU(cell_size, return_sequences=True, return_state=True) (input_data_a)
    
    encoder_outputs_1, rnn_layer_1 = GRU(cell_size, return_sequences=True, return_state=True) (encoder_outputs)    
   
    rnn_layer = concatenate([rnn_layer, rnn_layer_1])
    
    output_data = Reshape([10,1])(output_data_i)
    
    output_data = concatenate([output_data, emb_output_id])

    output_data = Dense(2*cell_size, activation='relu')(output_data)

    rnn_layer_multi = Multiply()([output_data, rnn_layer])
    
    rnn_layer_reshape = Reshape([1,2*cell_size])(rnn_layer)
    
    rnn_layer_reshape = Lambda(tile_tile)(rnn_layer_reshape)

    output_result = concatenate([rnn_layer_reshape, rnn_layer_multi, output_data])
    
    output_rnn_layer = Bidirectional(GRU(cell_size, return_sequences=True))(output_result)
    
    output_rnn_layer_2 = Bidirectional(GRU(cell_size, return_sequences=True))(output_rnn_layer)
    
    output_rnn_layer_3 = Bidirectional(GRU(200, return_sequences=True))(output_rnn_layer_2)
    
    output_rnn_layer_4 = Bidirectional(GRU(200, return_sequences=True))(output_rnn_layer_3)
    
    output_result = concatenate([output_result, output_rnn_layer, output_rnn_layer_2, output_rnn_layer_3, output_rnn_layer_4])

    output_result = Dense(1000, activation='relu')(output_result)
    output_result_1 = Dense(784, activation='relu')(output_result)
    #output_result_2 = Dense(784, activation='elu')(output_result_1)
    
    #output_result = concatenate([output_result, output_result_1, output_result_2])
    
    output_result = Dropout(0.2)(output_result_1)

    output_result = Dense(11, activation='sigmoid')(output_result)
    
    #output_result = Reshape([10])(output_result)


    model = Model(inputs=[input_data_context_id, input_data_start_id,
                          input_data_end_id, input_data, output_data_i, input_id, output_id], outputs=output_result)

    sgd = Adam(lr=0.0008)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['binary_crossentropy', 'binary_accuracy'])
    model.summary()
    return model

checkpoint = ModelCheckpoint('C:/Users/13528/Task 1/Data/skip_net_glove_max_mtsk_more_layer_best_v6.hdf5', monitor='val_binary_accuracy', verbose=1, save_best_only=True, mode='auto')

#K.clear_session()

model = skip_model_5_mtsk_att()

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=4, 
                                   verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)

# model.fit_generator(train_generate(), steps_per_epoch=3000, epochs=300, verbose=1,
#                         validation_data=(X_valid,Y_valid),max_q_size=35,
#                       callbacks = [mstk_ndcg_callback(),reduceLROnPlat,checkpoint])

# model.fit_generator(train_generate(), steps_per_epoch=3000, epochs=300, verbose=1,
#                         validation_data=(X_valid,Y_valid),
#                       callbacks = [mstk_ndcg_callback(),reduceLROnPlat,checkpoint])
# model.fit(train_generate(), steps_per_epoch=3000, epochs=300, verbose=1,
#                         validation_data=(X_valid,Y_valid),
#                       callbacks = [mstk_ndcg_callback(),reduceLROnPlat,checkpoint])
model.fit(train_generate(), steps_per_epoch=100, epochs=5, verbose=1,
                        validation_data=(X_valid,Y_valid),
                      callbacks = [mstk_ndcg_callback(),reduceLROnPlat,checkpoint])
model.save("skip_net_glove_max_mtsk_more_layer_best_v6.h5")

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_id (InputLayer)          [(None, 10)]         0           []                               
                                                                                                  
 context (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 start (InputLayer)             [(None, 10)]         0           []                               
                                                                                                  
 end (InputLayer)               [(None, 10)]         0           []                               
                                                                                            

 dense_11 (Dense)               (None, 10, 11)       8635        ['dropout_2[0][0]']              
                                                                                                  
Total params: 679,216,725
Trainable params: 15,773,094
Non-trainable params: 663,443,631
__________________________________________________________________________________________________


C:\Users\13528\AppData\Local\Temp/ipykernel_11360/2914680234.py:52: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_data = raw_data.astype(np.float)
C:\Users\13528\AppData\Local\Temp/ipykernel_11360/2914680234.py:57: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdoc

Epoch 1/5
100/100 [==============================] - ETA: 0s - loss: 0.3224 - binary_crossentropy: 0.3224 - binary_accuracy: 0.85090.7073958186305612

Epoch 1: val_binary_accuracy improved from -inf to 0.85411, saving model to C:/Users/13528/Task 1/Data\skip_net_glove_max_mtsk_more_layer_best_v6.hdf5
100/100 [==============================] - 2002s 20s/step - loss: 0.3224 - binary_crossentropy: 0.3224 - binary_accuracy: 0.8509 - val_loss: 0.3223 - val_binary_crossentropy: 0.3223 - val_binary_accuracy: 0.8541 - lr: 8.0000e-04
Epoch 2/5
100/100 [==============================] - ETA: 0s - loss: 0.3021 - binary_crossentropy: 0.3021 - binary_accuracy: 0.8626 0.7118999784982264

Epoch 2: val_binary_accuracy improved from 0.85411 to 0.85576, saving model to C:/Users/13528/Task 1/Data\skip_net_glove_max_mtsk_more_layer_best_v6.hdf5
100/100 [==============================] - 2026s 20s/step - loss: 0.3021 - binary_crossentropy: 0.3021 - binary_accuracy: 0.8626 - val_loss: 0.3167 - val_binary_cr

InvalidArgumentError: Graph execution error:

Detected at node 'model_2/embedding_10/embedding_lookup' defined at (most recent call last):
    File "C:\Users\13528\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\13528\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\13528\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\13528\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\13528\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\13528\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\13528\anaconda3\lib\asyncio\base_events.py", line 596, in run_forever
      self